In [3]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")
df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [ ]:

result = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)
top_city = result.iloc[0]
print(top_city)


city               Chennai
total_amount    1080909.79
Name: 1, dtype: object


In [7]:
result2 = (
    df.groupby("cuisine")["total_amount"].mean().reset_index().sort_values(by="total_amount", ascending=False)
)
result2

,cuisine,total_amount
3,Mexican,808.021344
2,Italian,799.448578
1,Indian,798.466011
0,Chinese,798.389020


In [11]:
result3=(df.groupby("user_id")["total_amount"].sum().reset_index())
count=(result3["total_amount"]>1000).sum()
count

np.int64(2544)

In [16]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"]=pd.cut(df['rating'],bins=bins,labels=labels,include_lowest=True)

result4=(df.groupby('rating_range')['total_amount']).sum().reset_index().sort_values(by="total_amount",ascending=False)
result4

C:\Users\omdon\AppData\Local\Temp\ipykernel_18416\3615270461.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result4=(df.groupby('rating_range')['total_amount']).sum().reset_index().sort_values(by="total_amount",ascending=False)


,rating_range,total_amount
3,4.6–5.0,2197030.75
0,3.0–3.5,2136772.70
2,4.1–4.5,1960326.26
1,3.6–4.0,1717494.41


In [17]:
result5=(df[df["membership"]=="Gold"].groupby('city')["total_amount"].mean().reset_index().sort_values(by='total_amount',ascending=False))
result5

,city,total_amount
1,Chennai,808.459080
2,Hyderabad,806.421034
0,Bangalore,793.223756
3,Pune,781.162243


In [21]:
result6 = (
    df.groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .reset_index()
    .sort_values(by="distinct_restaurants")
)

result6

,cuisine,distinct_restaurants,total_revenue
0,Chinese,120,1930504.65
1,Indian,126,1971412.58
2,Italian,126,2024203.80
3,Mexican,128,2085503.09


In [22]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

result7 = round((gold_orders / total_orders) * 100)
result7

50

In [25]:
result8 = (
    df.groupby(["restaurant_id", "restaurant_name_x"])
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
    .reset_index()
)

filtered = result8[result8["total_orders"] < 20].sort_values(
    by="avg_order_value", ascending=False
)

filtered


,restaurant_id,restaurant_name_x,total_orders,avg_order_value
293,294,Hotel Dhaba Multicuisine,13,1040.222308
261,262,Hotel Cafe Punjabi,18,1029.473333
76,77,Sri Mess Punjabi,12,1029.180833
192,193,Swagath Eatery Multicuisine,15,1026.306667
6,7,Ruchi Biryani Punjabi,16,1002.140625
...,...,...,...,...
183,184,Annapurna Tiffins Punjabi,19,621.828947
497,498,Darbar Tiffins Non-Veg,18,596.815556
191,192,Darbar Restaurant Punjabi,14,589.972857
328,329,Spice Tiffins Pure Veg,15,578.578667


In [26]:
result9 = {
    "Gold + Indian cuisine": df[(df["membership"]=="Gold") & (df["cuisine"]=="Indian")]["total_amount"].sum(),
    "Gold + Italian cuisine": df[(df["membership"]=="Gold") & (df["cuisine"]=="Italian")]["total_amount"].sum(),
    "Regular + Indian cuisine": df[(df["membership"]=="Regular") & (df["cuisine"]=="Indian")]["total_amount"].sum(),
    "Regular + Chinese cuisine": df[(df["membership"]=="Regular") & (df["cuisine"]=="Chinese")]["total_amount"].sum(),
}

result9

{'Gold + Indian cuisine': np.float64(979312.31),
 'Gold + Italian cuisine': np.float64(1005779.05),
 'Regular + Indian cuisine': np.float64(992100.27),
 'Regular + Chinese cuisine': np.float64(952790.9099999999)}

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"])

df["quarter"] = df["order_date"].dt.quarter

result10 = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

result10

C:\Users\omdon\AppData\Local\Temp\ipykernel_18416\1231849479.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,quarter,total_amount
2,3,2037385.10
3,4,2018263.66
0,1,2010626.64
1,2,1945348.72


In [30]:
result11=(df[df['membership']=='Gold'].shape[0])
result11

4987

In [31]:
result12=(df[df['city']=='Hyderabad']['total_amount'].sum())
result12

np.float64(1889366.58)

In [32]:
result13 = df["user_id"].nunique()
result13

2883

In [33]:
result14 = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)
result14

np.float64(797.15)

In [34]:
result15 = df[df["rating"] >= 4.5].shape[0]
result15

3374

In [35]:
gold_df = df[df["membership"] == "Gold"]


result16 = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

order_count = gold_df[gold_df["city"] == result16].shape[0]

result16, order_count

('Chennai', 1337)